# Band Spectrum

Calculate the band spectra displayed in Figures 2,3,4 and export as csv.

In [ ]:
# Run this cell once at startup

l = var('l')
E = var('E')

def mon(v):
#
# Calculate the general monodromy matrix with scaling l #
# and energy E
#
# Example:
# v = [1,1,0]; mon(v).expand()
# > [E^2 - E*l - 1            -E]
#   [        E - l            -1]
#
##########################
    M = identity_matrix(2)
    for vv in v:
        M = matrix(2,2,[E - l*vv, -1, 1, 0])*M
    return M

def lim_pots(v):
#
# determine all unique limit potentials of a given
# periodic potential v by considering all shifts and flips
#
##########################
    v_rev = deepcopy(v)
    lim = shift_pots(v)
    v_rev.reverse()
    lim_rev = shift_pots(v_rev)
    for pot in lim_rev:
        if pot not in lim:
            lim.append(pot)
    return lim
    
def shift_pots(v):
#
# determine all unique shifts of a given periodic potential v
#
##########################
    shift = []
    for c in range(len(v)):
        if v not in shift:
            shift.append(v)
        v = right_shift(v)
    return shift
    
def right_shift(a):
    return [a[-1] , *a[:-1]]

def cutoff(v,K):
#
# (deprecated, everything can also be done with `mon`)
#
# determine the cutoff matrix H_{0..K} with potential v[0:K-1]
#
##########################
    A = matrix(SR,K+1,K+1) #start enumerating from 0
    A = A + matrix.toeplitz([0,1] + [0]*(K-1), [1]+[0]*(K-1))
    for k in range(K+1):
        A[k,k] = v[k]
    return A

In [ ]:
def determine_band_boundaries(v):
    tr = mon(v).trace()
    sols = solve(tr^2 == 4, E)
    return(sols)

def evaluate_bands(sols,nsteps=150,start=0,end=1.5):
    bands = {}
    for s in range(nsteps + 1):
        ll = start + end * s/nsteps
        bands[ll] = list(map(lambda sol: real(sol.rhs().subs(l=ll).n()), sols))
        bands[ll].sort()
    return(bands)

def export_bands_csv(bands, outname='bands.csv'):
#
# Export band boundaries to csv file. 
# This file can be processed by further 
# routines in order to generate TikZ compatible code.
#
    f = open(outname, 'w')
    writefile = csv.writer(f)
    for b in bands:
        writefile.writerow([b, len(bands[b])/2,*bands[b]])
    f.close()

# K4 

Here we consider the potential $v= ( 1,1,0,1)$ and first calculate the boundaries of the bands.

In [ ]:
v = (1,1,0,1)
sols = determine_band_boundaries(v)
bands = evaluate_bands(sols)
export_bands_csv(bands, 'K4_bands.csv')

# K5

Evaluate bands for $v = (1,1,0,1,0)$

In [ ]:
v = (1,1,0,1,0)
sols = determine_band_boundaries(v)
bands = evaluate_bands(sols)
export_bands_csv(bands, 'K5_bands.csv')

# K9

Evaluate bands for $v = (1, 1, 0, 1, 0, 1, 0, 1, 1)$

In [ ]:
def determine_band_boundaries_num(v,ll=1):
    tr = mon(v).trace()
    sols = solve(tr.subs(l=ll)^2 == 4, E, to_poly_solve=True)
    return(sols)

In [ ]:
v = (1,1,0,1,0,1,0,1,1)
tr = mon(v).trace()
bands = {}
nsteps =250
start = 0
end = 2.5

for s in range(nsteps + 1):
        ll = start + end * s/nsteps
        sols = determine_band_boundaries_num(v,ll)
        bands[ll] = list(map(lambda sol: real(sol.rhs().n()), sols))
        bands[ll].sort()
        
export_bands_csv(bands, 'K9_bands.csv')